In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

from src.config.models import DatabaseConfig, LLMConfig, PromptConfig

db_config = DatabaseConfig(
    type="postgres",
    host="localhost",
    port="5432",
    database="postgres",
    user="postgres",
    password=os.getenv("POSTGRES_PWD"),
    schema="video_games"
)

In [2]:
from src.connettori.postgres import PostgresManager

db = PostgresManager(
    host=db_config.host,
    port=db_config.port,
    database=db_config.database,
    user=db_config.user,
    password=db_config.password
)
db.connect()

True

In [3]:
from src.dbcontext.postgres_metadata_retriever import PostgresMetadataRetriever

metadata_retriever = PostgresMetadataRetriever(db.engine, schema=db_config.schema)

print(metadata_retriever.get_table_definition("genre"))

CREATE TABLE genre (id integer NOT NULL, genre_name character varying(50));


In [4]:
from src.llm_handler.ollama_handler import OllamaHandler

llm = OllamaHandler(
    base_url="http://localhost:11434",
    model="llama3.1"
)

In [6]:
response = llm.get_completion("raccontami una storia (breve)")
print(response)

Era una notte di luna piena a Venezia. Un giovane pittore, Luca, si trovava sulla riva del canale Grande, con il pennello e la tavolozza in mano. Era ossessionato da un sogno: dipingere la città eterna nella sua forma più bella.

Mentre guardava le luci delle case e dei negozi che rifluivano sull'acqua, vide una figura femminile emergere dalle ombre. Era una donna con i capelli lunghi e il vestito azzurro come il cielo notturno. Si avvicinò a Luca e gli disse: "Voglio essere la tua musa per questa notte".

Luca, senza esitare, iniziò a dipingere. La donna si posizionò davanti a lui, con le braccia aperte come se stesse volando. Il pennello di Luca danzava sulla tela, lasciando un'immagine sempre più bella e struggente.

Quando la notte era quasi finita, la donna scomparve nel nulla, lasciando Luca solo con il suo dipinto. Era una scena meravigliosa: le case di Venezia riflettevano sulla superficie dell'acqua come se fossero fatte d'argento e d'oro.

Luca guardò il quadro, poi si voltò 

In [7]:
from src.llm_input.prompt_generator import PromptGenerator

prompt_config = PromptConfig(include_sample_data=True, max_sample_rows=3)
prompt_generator = PromptGenerator(
    metadata_retriever=metadata_retriever,
    schema_name=db_config.schema,
    prompt_config=prompt_config
)

In [8]:
user_question = "Mostrami i primi 5 giochi del database"
prompt = prompt_generator.generate_prompt(user_question)
print(prompt)

You are an SQL expert who helps convert natural language queries into SQL queries.
Your task is:
1. Generate a valid SQL query that answers the user's question
2. Provide a brief explanation of the results

You must respond with a JSON object in the following format:
{
    "query": "YOUR SQL QUERY HERE",
    "explanation": "Brief explanation of what the query does and what results to expect"
}

Important:
- Always insert schema name "video_games" before the tables
- Do not include comments in the SQL query
- The query must be executable
- Use the table DDL information to ensure correct column names and types
- Follow the foreign key relationships when joining tables
- If you do not have the necessary information to respond or if the requested data does not appear to be in the DB:
    - Explain in the explanation field why the request cannot be fulfilled
    - generate a simple SQL query to extract generic data from a single table (with a limit 5)
    - Explain what the sample data show

In [ ]:
response = llm.get_completion(prompt)

In [10]:
from src.llm_output.response_handler import ResponseHandler
response_handler = ResponseHandler(db, db_config.schema)
results = response_handler.process_response(response)

if results["success"]:
    print("Query SQL:", results["query"])
    print("\nSpiegazione:", results["explanation"])
    print("\nRisultati:", results["results"])

Query SQL: SELECT * FROM video_games.game LIMIT 5

Spiegazione: Questa query restituisce i primi 5 record della tabella game. La limitazione è impostata a 5 per mostrare solo i primi 5 giochi del database.

Risultati: [{'id': 1, 'genre_id': 11, 'game_name': "'98 Koshien"}, {'id': 2, 'genre_id': 8, 'game_name': '.hack//G.U. Vol.1//Rebirth'}, {'id': 3, 'genre_id': 8, 'game_name': '.hack//G.U. Vol.2//Reminisce'}, {'id': 4, 'genre_id': 8, 'game_name': '.hack//G.U. Vol.2//Reminisce (jp sales)'}, {'id': 5, 'genre_id': 8, 'game_name': '.hack//G.U. Vol.3//Redemption'}]
